In [1]:
# https://www.tensorflow.org/tutorials/layers
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

In [2]:
shuffle = np.load('data.npy')

In [3]:
from scipy.io import loadmat
data = loadmat("letters_data.mat")
train_x = data['train_x']

# y needs to be one hot encoded
old_y = data['train_y'] - 1
train_y = np.zeros((len(old_y), 26))
for i in range(len(old_y)):
    train_y[i, old_y[i]] = 1
tx = train_x[shuffle][:100000]
ty = train_y[shuffle][:100000]
vx = train_x[shuffle][100000:]
vy = train_y[shuffle][100000:]

In [4]:
def batch_generator(n):
    while True:
        shuffle = np.random.choice(np.arange(100000), replace=False, size=100000)
        for i in range(1000):
            yield tx[i * n: (i+1) * n], ty[i * n: (i+1) * n]

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y_ = tf.placeholder(tf.float32, shape=[None, 26], name='y_')

In [6]:
input_layer= tf.reshape(x, [-1, 28, 28, 1], name='input')

In [7]:
# Conv Pool 1
conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu,
      name='conv1')
pool1 = tf.layers.max_pooling2d(
    inputs=conv1,
    pool_size=[2, 2],
    strides=2,
    name='pool1')

In [8]:
# Conv Pool 2
conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu,
      name='conv2')
pool2 = tf.layers.max_pooling2d(
    inputs=conv2,
    pool_size=[2, 2],
    strides=2,
    name='pool2')

In [11]:
# Dense layer 1
keep_prob = tf.placeholder(tf.float32)
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64], name='pool2_flat')
dense1 = tf.layers.dense(
    inputs=pool2_flat,
    units=1024,
    activation=tf.nn.relu,
    name='dense1')
dropout1 = tf.layers.dropout(dense1, name='dropout1', rate=keep_prob)


TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

In [10]:
# Dense layer 2
dense2 = tf.layers.dense(
    inputs=dropout1,
    units=512,
    activation=tf.nn.relu,
    name='dense2')
dropout2 = tf.layers.dropout(dense2, name='dropout2', rate=keep_prob)

In [11]:
# Logits
logits = tf.layers.dense(inputs=dropout2, units=26, name='logits')

In [12]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
# restore
"""
new_saver = tf.train.import_meta_graph('neural-net-model.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('./'))
all_vars = tf.get_collection('vars')
for v in all_vars:
    v_ = sess.run(v)
    print(v_)"""

"\nnew_saver = tf.train.import_meta_graph('neural-net-model.meta')\nnew_saver.restore(sess, tf.train.latest_checkpoint('./'))\nall_vars = tf.get_collection('vars')\nfor v in all_vars:\n    v_ = sess.run(v)\n    print(v_)"

In [14]:
all_vars

NameError: name 'all_vars' is not defined

In [15]:
# new instance
sess.run(tf.global_variables_initializer())
batch_gen = batch_generator(100)

In [16]:
for i in range(100000):        
    batch = next(batch_gen)
    if i%1000 == 0:
        print 'test_accuracy', accuracy.eval(feed_dict={x: vx, y_: vy})
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1]})
        print 'step', i, 'training accuracy', train_accuracy
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: vx, y_: vy}))

test_accuracy 0.0504032
step 0 training accuracy 0.05
step 100 training accuracy 0.65
step 200 training accuracy 0.6
step 300 training accuracy 0.76
step 400 training accuracy 0.76
step 500 training accuracy 0.85
step 600 training accuracy 0.87
step 700 training accuracy 0.87
step 800 training accuracy 0.77
step 900 training accuracy 0.89
test_accuracy 0.887903
step 1000 training accuracy 0.92
step 1100 training accuracy 0.93
step 1200 training accuracy 0.91
step 1300 training accuracy 0.86
step 1400 training accuracy 0.82
step 1500 training accuracy 0.93
step 1600 training accuracy 0.89
step 1700 training accuracy 0.97
step 1800 training accuracy 0.86
step 1900 training accuracy 0.96
test_accuracy 0.910323
step 2000 training accuracy 0.92
step 2100 training accuracy 0.95
step 2200 training accuracy 0.93
step 2300 training accuracy 0.94
step 2400 training accuracy 0.86
step 2500 training accuracy 0.93
step 2600 training accuracy 0.92
step 2700 training accuracy 0.97
step 2800 training 

test_accuracy 0.931371
step 46000 training accuracy 1.0
step 46100 training accuracy 0.99
step 46200 training accuracy 1.0
step 46300 training accuracy 0.99
step 46400 training accuracy 0.99
step 46500 training accuracy 1.0
step 46600 training accuracy 1.0
step 46700 training accuracy 1.0
step 46800 training accuracy 1.0
step 46900 training accuracy 1.0
test_accuracy 0.932782
step 47000 training accuracy 1.0
step 47100 training accuracy 1.0
step 47200 training accuracy 1.0
step 47300 training accuracy 0.99
step 47400 training accuracy 1.0
step 47500 training accuracy 1.0
step 47600 training accuracy 1.0
step 47700 training accuracy 0.99
step 47800 training accuracy 0.99
step 47900 training accuracy 1.0
test_accuracy 0.933185
step 48000 training accuracy 1.0
step 48100 training accuracy 1.0
step 48200 training accuracy 1.0
step 48300 training accuracy 0.99
step 48400 training accuracy 1.0
step 48500 training accuracy 1.0
step 48600 training accuracy 1.0
step 48700 training accuracy 1.0


step 69100 training accuracy 0.99
step 69200 training accuracy 1.0
step 69300 training accuracy 0.99
step 69400 training accuracy 1.0
step 69500 training accuracy 1.0
step 69600 training accuracy 1.0
step 69700 training accuracy 1.0
step 69800 training accuracy 0.99
step 69900 training accuracy 1.0
test_accuracy 0.933669
step 70000 training accuracy 1.0
step 70100 training accuracy 1.0
step 70200 training accuracy 1.0
step 70300 training accuracy 0.99
step 70400 training accuracy 1.0
step 70500 training accuracy 1.0
step 70600 training accuracy 1.0
step 70700 training accuracy 1.0
step 70800 training accuracy 1.0
step 70900 training accuracy 0.97
test_accuracy 0.930766
step 71000 training accuracy 1.0
step 71100 training accuracy 1.0
step 71200 training accuracy 1.0
step 71300 training accuracy 1.0
step 71400 training accuracy 1.0
step 71500 training accuracy 0.99
step 71600 training accuracy 1.0
step 71700 training accuracy 1.0
step 71800 training accuracy 1.0
step 71900 training accu

step 92300 training accuracy 1.0
step 92400 training accuracy 1.0
step 92500 training accuracy 1.0
step 92600 training accuracy 1.0
step 92700 training accuracy 1.0
step 92800 training accuracy 1.0
step 92900 training accuracy 1.0
test_accuracy 0.932137
step 93000 training accuracy 1.0
step 93100 training accuracy 1.0
step 93200 training accuracy 1.0
step 93300 training accuracy 1.0
step 93400 training accuracy 1.0
step 93500 training accuracy 1.0
step 93600 training accuracy 1.0
step 93700 training accuracy 1.0
step 93800 training accuracy 0.98
step 93900 training accuracy 1.0
test_accuracy 0.932379
step 94000 training accuracy 1.0
step 94100 training accuracy 1.0
step 94200 training accuracy 0.99
step 94300 training accuracy 1.0
step 94400 training accuracy 1.0
step 94500 training accuracy 1.0
step 94600 training accuracy 0.99
step 94700 training accuracy 1.0
step 94800 training accuracy 1.0
step 94900 training accuracy 1.0
test_accuracy 0.931814
step 95000 training accuracy 1.0
step

In [14]:
# save
variables = [
    x,
    y_,
    input_layer,
    conv1,
    pool1,
    conv2,
    pool2,
    keep_prob,
    pool2_flat,
    dense1,
    dropout1,
    dense2,
    dropout2,
    logits
]
for var in variables:
    tf.add_to_collection('vars', var)
saver = tf.train.Saver()
saver.save(sess, 'neural-net-model')

'neural-net-model'